Imports

In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np

TF-IDF class. Contains methods which can easily access relevant information from the csv file which the class reads, which represents the TF-IDF model.

TF-IDF stands for Term Frequency - Inverse Document Frequency. It gives each keyword-document combination a weight, based on how often it occurs in the document, but also based on how often it occurs in all documents. This ensures that words which occur often in many documents do not receive high weights, rather only words which are relatively unique to a document and occur frequently in that document get high weights.

If all documents are about topic A, we do not care that document x contains is about document A.

Implementation wise, the TF-IDF is a csv file with each row representing a document and each column representing a keyword, the value at i,j represents the weight that keyword j has in document i.

In [2]:
class TfIdf:
    def __init__(self):
        # Loading in the tf-idf
        # Rows are documents, the first column is the document id
        # Columns are keywords, the first row is the keyword id
        self.tfidf = pd.read_csv("../data/tf_idf/tf_idf_time_weighted.csv")

        # Loading in the keywords
        # Two columns, column 1 is id, column 2 is keyword
        # We can access the weight in the tf-idf by first accessing the id number from the keyword file
        self.tfidf_keywords = pd.read_csv("../data/tf_idf/tf_idf_keywords_new.csv")
        self.tfidf_keywords.columns = ["id", "keyword"]

    def get_id_by_keyword(self, keyword):
        return self.tfidf_keywords.id.iloc[self.tfidf_keywords[self.tfidf_keywords.keyword == keyword].index].values[0]


    def get_keyword_by_id(self, id):
        return self.tfidf_keywords.loc[self.tfidf_keywords.id==id, 'keyword'].iloc[0]
    # Method to get the tfidf value of a keyword in a page
    def get_tf_idf_value(self, page_id, keyword):
        keyword_id = self.get_id_by_keyword(keyword)
        return self.tfidf.iloc[page_id-1, keyword_id + 1]

    def get_all_keywords_by_id(self, page_id):
        keywords = self.tfidf.iloc[page_id-1][1:].values
        return keywords

    def get_all_keywords_by_id_normalized(self, page_id):
        keywords_weights = self.get_all_keywords_by_id(page_id)
        sum_value = sum(keywords_weights)
        if sum_value == 0:
            return keywords_weights
        keywords_weights_normalized = keywords_weights/sum_value
        return keywords_weights_normalized

    def get_number_of_keywords(self):
        return self.tfidf_keywords.shape[0]

Instantiate the object

In [3]:
tfidf = TfIdf()

Read in the dataset of visits, used to test the information need prediction.

In [4]:
generic = lambda x: literal_eval(x)

conv = {'url_id_path': generic,
        'seconds_spent_path': generic}
df = pd.read_csv('data/clickdata/dataNoUnscrapedVisitsOrUnder20Sec.csv', converters=conv)

Create seperate dataframes for the paths taken and for the seconds spent on each url.
In paths, index 0 gives a list of urls, the equivalent index 0 in seconds represents how many seconds the user spent on the equal index urls.

ATTENTION!!!!!!!!!!!!!!!!!!!!


In [5]:
paths = df.url_id_path
secs = df.seconds_spent_path


print(len(paths))

3583310


In [6]:
list = tfidf.get_all_keywords_by_id(1)
for i in range(0, len(list)):
    if list[i]!=0:
        print(i)

3534
8038
8983


)Simplified implementation of IUNIS, which simply looks at the keywords which the user comes across, does not use the adjacency matrix, proximal cues, and spreading activation.
It seems that these are mostly important for WUFIS, which we are not interested in, thus implementing it is not worth the effort.

Input is the ordered list of page id's that the user has visited.
For each page, take the TF-IDF values from that page, which is a row of keywords, with either 0 if the keyword is not present or [0,1] if it is.

Then, for each next page, we add the TF-IDF values onto the existing weights.

Eventually we can sort the list and we will have the keywords with the highest weight on top, which then represents the most relevant keywords in the visit up until now, and acts as our prediction for what the user is looking for.
Instead of only looking at the user alone, we can also compare their information need to information needs of other users, and by looking at where the other users end up, we can predict where our user might want to end up.

Optionally, a decay factor can be introdced. Essentially, every iteration, all values are divided by a factor, meaning that keywords on recent pages are more biased and weighted more heavily.

It is also possible to give more weight to pages based on how much time the user spent on that page.

In [7]:
from pandas import DataFrame
#Method which essentially just takes the keywords and their weights on each page the user visits, continues to the next url, and then sums the weights.
#Be sure to realise that only passing a path will execute the normal implementation, which simply sums all new tf-idf values of each url.
#This is because the default decay_factor value is 1, meaning that nothing changes, and the default secs is None, if that is None, the if statement with secs is never entered and we do not take time into account.
#Sorted gives an option to sort the vector before returning, this puts the most relevant keywords on top, but does not allow for easy similarity comparison, thus should only be used if this is not the intention
def find_keyword_weights(path, decay_factor=1, secs=None, sorted=False):
    #initiate array of weights (weights start as 0, size=num of keywords)
    weights = np.zeros(tfidf.get_number_of_keywords())
    #ensures all output is visible
    pd.options.display.max_rows = 0
    #Iterate over each page url
    for i in range(len(path)):
        #Divide the weights by the decay factor (1 by default, no decay in that case)
        weights = weights/decay_factor
        #Initialize the tf-idf weights of the current page
        new_weights = tfidf.get_all_keywords_by_id_normalized(path[i])
        #If we are taking time into account, multiply the weights by the amount of time spent on this page
        if secs is not None:
            new_weights*=secs[i]
        #Add the new weights to the existing weights
        weights += new_weights
        #Normalize such that the largest value becomes 1
        if max(weights)==0:
            return weights
        weights /= max(weights)
    #Option to sort s.t. max weights are on top.
    if sorted:
        df = DataFrame(weights, columns=['weights'])
        return df.sort_values(by=['weights'], ascending=False)
    return weights

Method which calculates keywords of a visit

In [8]:
def create_dictionary(keys, values):
    result = {} # empty dictionary
    for key, value in zip(keys, values):
        if value != 0 and not np.isnan(value) and key != 'laxxxxxxxxx' and key!= 'boxxxx':
            result[key] = value
    return result

In [ ]:
def remove_keywordless_pages(path, seconds):
    indices_to_remove = []
    for i in range(len(path)):
        if path[i]>1494:
            indices_to_remove.append(i)
    for i in range(len(indices_to_remove)-1, -1, -1):
        index = indices_to_remove[i]
        path.pop(index)
        seconds.pop(index)
    return path, seconds

In [9]:
def compute_keywords_of_complete_visit(min_visit_size, paths):
    dicts = []
    keys = []
    for i in range(0, tfidf.get_number_of_keywords()):
        keys.append(tfidf.get_keyword_by_id(i))

    for i in range(len(paths)):
        path = paths[i]
        if i % 10000 == 0:
            print(i, 'out of', len(paths))
        path[:] = [x for x in path if x<=1555]
        if len(path)>=min_visit_size:
            weights = find_keyword_weights(path, decay_factor=7)
            keywords_weights = create_dictionary(keys, weights)
            id_need = (i, keywords_weights)
            dicts.append(id_need)
            
    return dicts

In [10]:
dicts = compute_keywords_of_complete_visit(5, paths)

0 out of 3583310


/var/folders/yv/gpc5sh7j5k3cls_fv_dpzntc0000gn/T/ipykernel_836/3435437501.py:32: RuntimeWarning: invalid value encountered in true_divide
  keywords_weights_normalized = keywords_weights/sum_value


10000 out of 3583310
20000 out of 3583310
30000 out of 3583310
40000 out of 3583310
50000 out of 3583310
60000 out of 3583310
70000 out of 3583310
80000 out of 3583310
90000 out of 3583310
100000 out of 3583310
110000 out of 3583310
120000 out of 3583310
130000 out of 3583310
140000 out of 3583310
150000 out of 3583310
160000 out of 3583310
170000 out of 3583310
180000 out of 3583310
190000 out of 3583310
200000 out of 3583310
210000 out of 3583310
220000 out of 3583310
230000 out of 3583310
240000 out of 3583310
250000 out of 3583310
260000 out of 3583310
270000 out of 3583310
280000 out of 3583310
290000 out of 3583310
300000 out of 3583310
310000 out of 3583310
320000 out of 3583310
330000 out of 3583310
340000 out of 3583310
350000 out of 3583310
360000 out of 3583310
370000 out of 3583310
380000 out of 3583310
390000 out of 3583310
400000 out of 3583310
410000 out of 3583310
420000 out of 3583310
430000 out of 3583310
440000 out of 3583310
450000 out of 3583310
460000 out of 35833

In [11]:
import json

In [12]:
with open('ids_keywords_weights.json', 'w') as outfile:
    json.dump(dicts, outfile)